In [3]:
#https://www.geeksforgeeks.org/extract-ip-address-from-file-using-python/

In [4]:
#Parse export from vCenter and create IP subnets table

import pprint as pp
import re
import ipaddress
#from mac_vendor_lookup import MacLookup

In [5]:
#Clean duplicate field from comma separated string
def cleandup(ip):
    iplist = ip.split(',')
    #return ','.join(list(dict.fromkeys(iplist)))
    return ','.join(list(set(iplist)))

In [6]:
#Return range of IP addressess for particular subnet
#takes first IP address if there are more IP's separated by coma
def getrange(ip, mask):
    try:
        ip = ip.split(',', 1)[0].strip()
        subnet = ipaddress.ip_network((ip, mask), strict = False)
        return str(subnet.network_address) + ' - ' +  str(subnet.broadcast_address)
    except (ValueError, TypeError):
        return('NO VALID IP')

In [7]:
#Change network format from XXX.YYY.ZZZ.NNN to the short notation with /XX 
def shortnet(ip, mask):
    try:
        ip = ip.split(',', 1)[0].strip()
        subnet = ipaddress.ip_network((ip, mask), strict = False)
        return str(subnet)
    except (ValueError, TypeError):
        return('NO VALID NETWORK')

In [8]:
def uniqueip(iplist, filename):
    ipstr = set(iplist)
    iplist = []
    #Convert IP strings to IpAddress in order to get sorting right
    for ip in ipstr:
        iplist.append(ipaddress.ip_address(ip))
    with open('SORTEDIP_' + filename, 'w', encoding='utf8') as output:
        for i in sorted(iplist):
            output.write(str(i) + '|' + filename + '\n')

In [9]:
fields = {'VM':None, 'Host':None, 'OS':None,
          'Tools':None, 'NicType':None, 'VLAN':None,
          'IP':None, 'Gateway':None, 'Subnet Mask':None,
          'Network':None, 'IP range':None,
          'DNS':None,'MAC':None, 'File': None}

header = ['VM', 'Host', 'OS', 'Tools', 'NicType', 'VLAN',
          'IP', 'Gateway', 'Subnet Mask', 'Network', 'IP range', 'DNS','MAC', 'File']

In [10]:
#tags to mark start and end block
starttag = 'VM'
endtag = 'MAC'
separator = '|'

#this is  list for holding ip's parsed from file
#other function will make unique sorted list
iplist = []

#set to true if looping inside start/end block
inside = False

filename = 'LV_VMs_Network.txt'
output = filename + '.csv'

with open(filename, encoding='utf8') as file, open(output, 'w', encoding='utf8') as output:
    #write csv header
    output.write(separator.join(header))
    output.write('\n')
    
    for row in file:
        #inside marker if row starts with start tag
        if row.startswith(starttag):
            inside = True
            
        #asign values to the keys while we are inside start-end block
        if inside:
            row_list = row.split(':', 1)
            
            #check if split have 2 fields, if only one- field continues on the next line
            if len(row_list) == 2:
                key = row_list[0].strip()
                value = row_list[1].strip()
                fields[key] = value
            #append rest of next line to the previous key value
            else:
                fields[key] = fields[key] + row.strip()
            
            #inside/outside status if endtag is hit
            if row.startswith(endtag):
                #Add calculated fields
                fields['IP range'] = getrange(fields['IP'], fields['Subnet Mask'])
                fields['Network'] = shortnet(fields['IP'], fields['Subnet Mask'])
                fields['File'] = filename
                if fields['IP'] != '':
                    iplist = iplist + fields['IP'].split(',')
                
                #Clean duplicates inside fields and write to file
                csv_string = ''
                for m in header:
                    csv_string = csv_string + cleandup(fields[m]).strip() + separator
                csv_string = csv_string + '\n'
                output.write(csv_string)
                #Set status to indicate we are outside start/end tags
                inside = False

#write unique IP's to the file
uniqueip(iplist, filename)

In [16]:
#Create summary subnets when given first and last IP address
def summarize_ip(ipfile):
    iplist = []
    with open(ipfile, encoding='utf8') as f:
        for line in f:
            iplist.append(ipaddress.ip_address(line.split('|')[0].strip()))
        #remove duplicates
        iplist = list(dict.fromkeys(iplist))
            
    ip_min = min(iplist)
    ip_max = max(iplist)
    
    print('MIN:', ip_min, 'MAX:', ip_max, '\n')
    for m in sorted(iplist):
        print(m)
    print()
    
    ip_range = ipaddress.summarize_address_range(ip_min, ip_max)
    for i in ip_range:
        #print(i, list(i.hosts()))
        print(i, ',', i.network_address, '-', i.broadcast_address)

In [1]:
#summarize_ip('SORTEDIP_LT_VMs_Network.txt')

In [12]:
#Read file with IP addressess and migration status (comma separated) and return dictionary
def read_ip(ipfile):
    iplist = {}
    with open(ipfile, encoding='utf8') as f:
        for line in f:
            pair = line.split('\t')
            key = pair[0].strip()
            value = pair[1].strip()
            iplist[key]= value
            #iplist.append(ipaddress.ip_address(line.strip()))
    return iplist

#take list of IP addressess and fit them into predefines net divided into subnets (offset)
iplist = read_ip('10.5.64.0.txt')
ipnetwork = '10.5.64.0/24'
offset = 2

subnets = list(ipaddress.ip_network(ipnetwork, strict = False).subnets(offset))
for net in subnets:
    
    network = net.network_address
    broadcast = net.broadcast_address
    
    print(net, str(network), '-', str(broadcast), '\n')
    for ip_s in iplist.keys():
        ip = ipaddress.ip_address(ip_s)
        if ip in net:
            #check if IP address is same as broadcast or subnet
            if ip == network or ip == broadcast:
                warn = '*'
            else:
                warn = ''
            print(ip_s, iplist[ip_s], warn)
    print()

FileNotFoundError: [Errno 2] No such file or directory: '10.5.64.0.txt'